In [1]:
import os
import pandas as pd
import numpy as np
import sys
import traceback
import pickle

#my libraries
src_path = 'path_to_src'
datar = 'path_to_data'
# add src_path if it's located in another path
#sys.path.append(os.path.join(src_path,'multimodal_retrieval/'))

### Reading data

In [2]:
datapath = os.path.join(datar,'datasets/mirflick25/gen/')

In [3]:
# val indexes
with open(os.path.join(datapath,'indexes.pkl'),'rb') as f:
    ind = pickle.load(f) #ind['train/val']
val_i = ind['val']
len(val_i)

864

In [4]:
# GT_samples-->{'2008_000716.jpg': 0,'2008_000716.txt': 0,
# ids_i -->1:'2008_000716.jpg',

In [5]:
import pickle
# reading resnet101 features 
data_f = os.path.join(datar,'datasets/mirflick25/gen/resnet_all.pkl')
with open(data_f,'rb') as f:
    images = pickle.load(f)
    
# correspondance between id and img code
with open(os.path.join(datapath,'idS_id_im.pkl'), 'rb') as f:
    ids_i = pickle.load(f)
    
images_f = {}
for idx in val_i:
    try:        
        images_f[ids_i.get(idx)] = images.get(idx)[0]
    except Exception as e:
        print ('CORRUPTED IMAGE: ',idx)
        print('Exception: %s',"".join(traceback.format_exception(*sys.exc_info())))
len(images_f)

829

In [6]:
np.unique([len(images_f.get(f)) for f in images_f.keys()])
#im4137
#im10590

array([2048])

In [7]:
import pickle
# reading resnet101 features
data_f = os.path.join(datar,'/datasets/mirflick25/gen2/bert_all.pkl')
with open(data_f,'rb') as f:
    texts = pickle.load(f)
len(texts)

# correspondance between id and txt code
with open(os.path.join(datapath,'idS_id_tx.pkl'), 'rb') as f:
    ids_t = pickle.load(f)

texts_f = {}
for idx in val_i:
    try:        
        texts_f[ids_t.get(idx)] = texts.get(idx)[0]
    except Exception as e:
        print ('CORRUPTED Text: ',s_id)
        print('Exception: %s',"".join(traceback.format_exception(*sys.exc_info())))
len(texts_f)

829

### Retrieval

In [8]:
#my libraries
from utils import eval_wiki_retrieval as ev_r#import calc_distances, get_AP   

# For retrieval
with open(os.path.join(datapath,'txt_img_cat.pkl'), 'rb') as f:
    GT_samples = pickle.load(f)

distances = ['corr','eu','cos']
topks = [10,100,200,len(images_f)-1]  

maps1 = ev_r.calc_distances(images_f,images_f,'img2img',topks,distances,GT_samples)
maps2 = ev_r.calc_distances(texts_f,texts_f,'txt2txt',topks,distances,GT_samples)
mapt_uni = pd.concat([maps1, maps2], axis=0)
mapt_uni = mapt_uni[['task','distances']+topks]
mapt_uni

,task,distances,10,100,200,828
0,img2img,corr,0.946286,0.897324,0.874730,0.793319
1,img2img,eu,0.937274,0.873944,0.844227,0.752887
2,img2img,cos,0.452237,0.422979,0.441741,0.554711
0,txt2txt,corr,0.870017,0.793998,0.764651,0.701716
1,txt2txt,eu,0.867265,0.786000,0.757452,0.699315
2,txt2txt,cos,0.708577,0.644884,0.637913,0.636745
